## Read in Collected Data and Process

In [33]:
%pip install scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [36]:
# Load the CSV file
df = pd.read_csv('fer2013_landmarks_nopathsfixed.csv')
df.head()
df.tail()
df = df.iloc[::2, :]
df = df.iloc[::2, :]
df = df.iloc[::2, :]

X = df.drop('emotion', axis=1) # features
y = df['emotion'] # target value
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
y_test

23144        sad
5168        fear
18776    neutral
5520        fear
6040        fear
          ...   
22064        sad
2816       angry
8376       happy
3472       angry
3864     disgust
Name: emotion, Length: 996, dtype: object

## Train Machine Learning Classification Model

In [37]:
# X = df.drop('emotion', axis=1).values
# y = df['emotion'].values

from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [38]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', max_iter=2000)),
    # 'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    # 'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    # 'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [39]:
fit_models = {}
for algo, pipeline in pipelines.items():
    print(algo, pipeline)
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

lr Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(max_iter=2000))])


In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [41]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.4979919678714859


In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
y_test

In [ ]:
with open('emotions.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## Encode Labels

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)
# # Split the data
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


## Define the model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# model = Sequential()
# model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
# # Evaluate the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
# # Make predictions
# predictions = model.predict(X_test)

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
data = pd.read_csv("fer2013_landmarks_nopathsfixed.csv")

# Data Preprocessing
# Assuming your CSV has columns 'emotion' and 'facial_coordinates'
# Handle missing values
data = data.dropna()
# Split the data into features (X) and labels (y)
# X = data['facial_coordinates'].apply(lambda x: np.fromstring(x[1:-1], sep=',', dtype=np.float32))
# y = data['emotion']
# Separate features (X) and target value (y)
X = data.drop('emotion', axis=1)
y = data['emotion']

# Label encode the emotions
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
# Convert to one-hot encoding
y_one_hot = to_categorical(y, num_classes=len(label_encoder.classes_))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.3, random_state=1234)

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[EarlyStopping(patience=3)])

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Print classification report and confusion matrix
print(classification_report(y_test_labels, y_pred_labels))
print(confusion_matrix(y_test_labels, y_pred_labels))




Epoch 1/10


465/465 [==============================] - 3s 5ms/step - loss: 1.7722 - accuracy: 0.2833 - val_loss: 1.7591 - val_accuracy: 0.2336
Epoch 2/10
465/465 [==============================] - 1s 3ms/step - loss: 1.6327 - accuracy: 0.3561 - val_loss: 1.6044 - val_accuracy: 0.4003
Epoch 3/10
465/465 [==============================] - 1s 3ms/step - loss: 1.5542 - accuracy: 0.3951 - val_loss: 1.5193 - val_accuracy: 0.4390
Epoch 4/10
465/465 [==============================] - 1s 3ms/step - loss: 1.5083 - accuracy: 0.4172 - val_loss: 1.5109 - val_accuracy: 0.4194
Epoch 5/10
465/465 [==============================] - 1s 3ms/step - loss: 1.4843 - accuracy: 0.4289 - val_loss: 1.4944 - val_accuracy: 0.4355
Epoch 6/10
465/465 [==============================] - 1s 3ms/step - loss: 1.4607 - accuracy: 0.4373 - val_loss: 1.4695 - val_accuracy: 0.4415
Epoch 7/10
465/465 [==============================] - 1s 2ms/step - loss: 1.4414 - accuracy: 0.4457 - val_loss: 1.4492 - val_accuracy: 0.4390
Ep

C:\Users\coldf\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\coldf\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\coldf\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
